In [1]:
from dotenv import dotenv_values
import requests

from helpers.google_interactor import GoogleInteractor
from helpers.html_parser import HTMLParser 
from helpers.storage_manager import StorageManager
from helpers.refiner_model import RefinerModel
from helpers.improver_model import ImproverModel
from helpers.writer_citer_model import WriterCiterModel



# Load environment variables
config = dotenv_values("../.env")
google_api_key = config['GOOGLE_API_KEY']
search_engine_id = config['SEARCH_ENGINE_ID']

# Instantiate helpers
google_interactor = GoogleInteractor(api_key=google_api_key, search_engine_id=search_engine_id)
storage_manager = StorageManager("../data/")    
website_refiner = RefinerModel(api_key=config['OPEN_AI_API_KEY'])
corpus_refiner = RefinerModel(api_key=config['OPEN_AI_API_KEY'])
improver_model = ImproverModel(api_key=config['OPEN_AI_API_KEY'])


# Load the clean corpus text
with open("../data/clean_content.txt") as f:
    corpus_text = f.read()

# Number of links to consider
K = 5

# Declare trial number
trial_number = 2

# Select a query to use for the current trial
query = "What products are offered by Southern Farm Bureau Life Insurance Company?"

# Submit the query to Google
search_results = google_interactor.search_google(query=query, num_results=10)
    
# Store the response
storage_manager.save_to_folder("InitialGoogleResponses", search_results, f"{trial_number}.json")


"""
Get Raw Text from the HTML associated with the Google Links
"""

# Retrieve the stored response
search_results = storage_manager.load_from_folder("InitialGoogleResponses", f"{trial_number}.json")

raw_text_from_html = []
items = search_results['items']

# For each item of raw text that was stored ...
for item in search_results['items'][:K]:
    link = item['link']

    # Fet the HTML associated with the link
    response = requests.get(link)

    # Get the raw text associated with the HTML
    parsed_text = HTMLParser.parse_html(response.content)

    # Store the raw text from the HTML
    raw_text_from_html.append({"link" : link, "raw_text" : parsed_text})
    
storage_manager.save_to_folder("RawTextFromHTML", raw_text_from_html, f"{trial_number}.json")


"""
Get the refined text from the raw text
"""

raw_text_from_html = storage_manager.load_from_folder("RawTextFromHTML", f"{trial_number}.json")

website_refiner_responses = []
# For each element of raw text that was stored ...
for current_item in raw_text_from_html:
    
    link = current_item['link']
    raw_text = current_item['raw_text']   

    
    response_from_website_refiner = website_refiner.refine_text(raw_text=raw_text, query=query)
    
    if response_from_website_refiner:
        response_from_website_refiner['link'] = link
        website_refiner_responses.append(response_from_website_refiner)
    else:
        print("Failed to refine text.")

storage_manager.save_to_folder("WebsiteRefinerModelResponses", website_refiner_responses, f"{trial_number}.json")


"""
Filter the corpus to information that may be relevant to the query
"""

response_from_corpus_refiner = corpus_refiner.refine_text(raw_text=corpus_text, query=query)
storage_manager.save_to_folder("CorpusRefinerModelResponses", response_from_corpus_refiner, f"{trial_number}.json")

Data saved to ../data/InitialGoogleResponses/2.json
Data loaded from ../data/InitialGoogleResponses/2.json
Data saved to ../data/RawTextFromHTML/2.json
Data loaded from ../data/RawTextFromHTML/2.json
Data saved to ../data/WebsiteRefinerModelResponses/2.json
Data saved to ../data/CorpusRefinerModelResponses/2.json


In [2]:
"""
Improve the text from the SFBLIC site
"""

# Get the refined text from the website
website_refiner_responses = storage_manager.load_from_folder("WebsiteRefinerModelResponses", f"{trial_number}.json")
sfblic_response = [response for response in website_refiner_responses if response['link'] == 'https://www.sfbli.com/'][0]
refined_text_from_website = sfblic_response['choices'][0]['message']['content']

# Get the refined text from the corpus
response_from_corpus_refiner = storage_manager.load_from_folder("CorpusRefinerModelResponses", f"{trial_number}.json")
refined_text_from_corpus = response_from_corpus_refiner['choices'][0]['message']['content']


# Generate improved text
improved_text = improver_model.improve_text(
    query=query,
    refined_text_from_website=refined_text_from_website,
    refined_text_from_corpus=refined_text_from_corpus
)

if improved_text:
    print("Improved Text generated...")
    
else:
    print("Failed to improve text.")

Data loaded from ../data/WebsiteRefinerModelResponses/2.json
Data loaded from ../data/CorpusRefinerModelResponses/2.json


'Southern Farm Bureau Life Insurance Company offers the following products:\n\n1. **Life Insurance Products**:\n   - **Permanent Life Insurance**: \n     - Whole life\n     - Single premium\n     - Modified premium\n     - Adjustable premium\n   - **Term Life Insurance**: Available in 10, 20, and 30-year intervals.\n\n2. **Financial Planning Services**:\n   - Estate and retirement planning\n   - Tax-qualified and non-qualified retirement plans\n   - Banking services (certificates of deposit, money market accounts, health savings accounts, loans)\n\n3. **Additional Services**:\n   - Customized policies for wealth transfer, debt repayment, and daily living costs.\n\nThe company operates in 11 states, emphasizing competitive products, superior customer service, and ethical practices.'

In [6]:
storage_manager.save_to_folder("ImproverModelResponses", improved_text, f"{trial_number}.json")

Data saved to ../data/ImproverModelResponses/2.json


In [8]:
improved_version = improved_text['choices'][0]['message']['content']
print(improved_version)

Southern Farm Bureau Life Insurance Company offers a range of products and services focused on providing competitive options for its policyholders. Their offerings include:

1. **Life Insurance Products**:
   - **Permanent Life Insurance**: Includes whole life, single premium, modified premium, and adjustable premium policies.
   - **Term Life Insurance**: Available in terms of 10, 20, and 30 years.

2. **Financial Planning Services**:
   - Estate and retirement planning
   - Tax-qualified and non-qualified retirement plans
   - Banking services such as certificates of deposit, money market accounts, health savings accounts, and loans.

3. **Additional Services**:
   - Customized policies designed for wealth transfer, debt repayment, and covering daily living costs.

The company operates in 11 states, prioritizing competitive products, superior customer service, and ethical practices. For more detailed information on specific products, contacting an agent or visiting the company's offi

In [12]:
website_refiner_model_responses = storage_manager.load_from_folder("WebsiteRefinerModelResponses", f"{trial_number}.json")

unmodified_set_of_reference_material =  [
    {
        "link" : item['link'],
        "information" : item['choices'][0]['message']['content']
    }
    for item in website_refiner_model_responses
]

unmodified_set_of_reference_material


Data loaded from ../data/WebsiteRefinerModelResponses/2.json


[{'link': 'https://www.sfbli.com/',
  'information': 'Southern Farm Bureau Life Insurance Company offers a range of life insurance products, focusing on competitive offerings for its policyholders. Specific products mentioned include:\n\n- Life Insurance\n- Health Plans\n- Behavioral Health Plans\n\nFor detailed product offerings, it is advisable to contact an agent or access their official website.'},
 {'link': 'https://w3.sfbcic.com/',
  'information': 'The provided text does not contain relevant information regarding the products offered by Southern Farm Bureau Life Insurance Company. Please provide a different source or query for assistance.'},
 {'link': 'https://www.sfbli.com/loginregister',
  'information': 'Southern Farm Bureau Life Insurance Company offers a range of life insurance products and services. Their mission focuses on providing competitive products and superior customer service to policyholders. Specific product offerings include:\n\n- Life insurance policies\n- Heal

In [13]:


# Initialize WriterCiterModel
writer_citer = WriterCiterModel(api_key=config['OPEN_AI_API_KEY'])

# Generate a cited response
cited_response = writer_citer.generate_cited_response(
    rephrased_query=query,
    set_of_reference_query=unmodified_set_of_reference_material
)

if cited_response:
    print("Cited Response:")
    print(cited_response)
else:
    print("Failed to generate cited response.")



Cited Response:
{'id': 'chatcmpl-AWqfwRIPPveotXd0luY8qK0cYc1hU', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': 'Southern Farm Bureau Life Insurance Company offers a variety of insurance products, primarily focusing on life insurance, health plans, and behavioral health plans[1][3]. Additionally, they provide products related to annuities for retirement savings and income[5]. For a broader range of offerings, their portfolio also includes farm insurance, business insurance, umbrella insurance, financial services, retirement planning, and estate planning[4]. To obtain detailed information on specific products and services, it is recommended to consult directly with the company or access their official website[1][3].', 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None}}], 'created': 1732392344, 'model': 'gpt-4o-2024-08-06', 'object': 'chat.completion', 'service_tier': None, 'system_fingerprint': 'fp_

In [18]:
new_set_of_reference_material = []

for item in unmodified_set_of_reference_material:
    if item['link'] == 'https://www.sfbli.com/':
        new_set_of_reference_material.append({"link" : link, "information" : improved_version})
    else:
        new_set_of_reference_material.append(item)

new_set_of_reference_material
        

[{'link': 'https://www.linkedin.com/company/southern-farm-bureau',
  'information': "Southern Farm Bureau Life Insurance Company offers a range of products and services focused on providing competitive options for its policyholders. Their offerings include:\n\n1. **Life Insurance Products**:\n   - **Permanent Life Insurance**: Includes whole life, single premium, modified premium, and adjustable premium policies.\n   - **Term Life Insurance**: Available in terms of 10, 20, and 30 years.\n\n2. **Financial Planning Services**:\n   - Estate and retirement planning\n   - Tax-qualified and non-qualified retirement plans\n   - Banking services such as certificates of deposit, money market accounts, health savings accounts, and loans.\n\n3. **Additional Services**:\n   - Customized policies designed for wealth transfer, debt repayment, and covering daily living costs.\n\nThe company operates in 11 states, prioritizing competitive products, superior customer service, and ethical practices. For

In [19]:
# Generate a cited response
cited_response = writer_citer.generate_cited_response(
    rephrased_query=query,
    set_of_reference_query=new_set_of_reference_material
)

if cited_response:
    print("Cited Response:")
    print(cited_response)
else:
    print("Failed to generate cited response.")

Cited Response:
{'id': 'chatcmpl-AWqjgOYPfaf6z9kFhdfgWLl9JTM5n', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': 'Southern Farm Bureau Life Insurance Company provides a variety of products and services designed to meet the needs of its policyholders. Their product offerings include a range of life insurance policies, featuring both permanent life insurance—such as whole life, single premium, modified premium, and adjustable premium policies—and term life insurance available in 10, 20, and 30-year terms [1]. Additionally, they offer health and behavioral health plans [3].\n\nThe company also provides various financial planning services including estate and retirement planning, tax-qualified and non-qualified retirement plans, and banking services such as certificates of deposit, money market accounts, health savings accounts, and loans [1]. Furthermore, they offer products like annuities for retirement savings and income, and have solutions for